In [ ]:
import pygame
from pygame.locals import *
import random
from OpenGL.GL import *
from OpenGL.GLU import *
import math
from IPython.display import clear_output
from operator import itemgetter
import numpy as np
import random


#leftRect = [(-5.125,0,-40),(-3.125,0,-40),(-3.125,5,-40),(-5.125,5,-40),]
#rightRect = [(3.125,0,-40),(5.125,0,-40),(5.125,5,-40),(3.125,5,-40)]
edges = ((0,1),(1,2),(2,3),(3,0))

fov_y = 45
width = 320
height = 240

def TransformVec3(vecA,mat44):
    vecB = [0, 0, 0, 0]
    for i0 in range(0, 4):
        vecB[i0] = vecA[0] * mat44[0*4+i0] + vecA[1] * mat44[1*4+i0] + vecA[2] * mat44[2*4+i0] + mat44[3*4+i0]
    return [vecB[0]/vecB[3], vecB[1]/vecB[3], vecB[2]/vecB[3]]

def TestRec(prjMat, ll):
    ll_ndc = TransformVec3(ll, prjMat)
    pos_pixel_xy = [width*(ll_ndc[0]+1.0)/2.0, height*(1.0-ll_ndc[1])/2.0]
    return (pos_pixel_xy)
def getCenter(right, left):
    x = (sum([x[0] for x in right])+sum([x[0] for x in left]))/8.0
    y = (sum([y[1] for y in right])+sum([y[0] for y in left]))/8.0
    return(x,y)
def getWidth(right, left):
    return (right[-1][0]-left[0][0])
def getHeight(right, left):
    return ((right[-1][1]+left[-1][1])-(right[0][1]+left[0][1]))/2.0
def getData(prjMat, right, left):
    leftCoordinates = [TestRec(prjMat,x) for x in left]
    rightCoordinates = [TestRec(prjMat,x) for x in right]
    center = getCenter(rightCoordinates, leftCoordinates)
    h = getHeight(sorted(rightCoordinates, key=itemgetter(1)),sorted(leftCoordinates,key=itemgetter(1)))
    w = getWidth(sorted(rightCoordinates, key=itemgetter(0)),sorted(leftCoordinates, key=itemgetter(0)))
    #print("Height"+str(sorted(rightCoordinates, key=itemgetter(1))))
    #print("Width"+str(sorted(rightCoordinates, key=itemgetter(0))))
    return [center, w, h]
#This draws the rectangles edges
def Target(rightRect,leftRect):

    prjMat = (GLfloat * 16)()
    glGetFloatv(GL_PROJECTION_MATRIX, prjMat)
    glBegin(GL_LINES)
    for edge in edges:
        for vertex in edge:
            glVertex3fv(leftRect[vertex])
    glEnd()

    glBegin(GL_LINES)
    for edge in edges:
        for vertex in edge:
            glVertex3fv(rightRect[vertex])
    glEnd()
def addToRow(right, left,index,adjust):
    for i in right:
        i[index] = i[index]+adjust
    for i in left:
        i[index] = i[index]+adjust
    return (right,left)
def main():
    leftRect = [[-5.125,0,-20],[-3.125,0,-20],[-3.125,5,-20],[-5.125,5,-20]]
    rightRect = [[3.125,0,-20],[5.125,0,-20],[5.125,5,-20],[3.125,5,-20]]
    try:
        pygame.init()
        display = (width,height)
        pygame.display.set_mode(display, DOUBLEBUF|OPENGL)
        glMatrixMode(GL_PROJECTION)
        gluPerspective(fov_y, (display[0]/display[1]), .1, 1000)
        glMatrixMode(GL_MODELVIEW)

        while True:
            #iterates through events to check for quits
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    quit() 
            Target(rightRect,leftRect)
            
            prjMat = (GLfloat * 16)()
            glGetFloatv(GL_PROJECTION_MATRIX, prjMat)
            data= getData(prjMat, rightRect, leftRect)
            leftArray = np.array(leftRect)
            leftPoint = leftArray.mean(axis=0)
            rightArray = np.array(rightRect)
            rightPoint = rightArray.mean(axis=0)
            print(rightPoint)
            toCSV = [data[0][0],data[0][1],data[1],data[2],leftPoint[0],leftPoint[1],leftPoint[2],rightPoint[0],rightPoint[1],rightPoint[2]]
            print(toCSV)
            
            ##set bounds and randomize input
            if(data[0][0] > 320 or data[0][0]<0 or data[0][1]>240 or data[0][1]<0 or leftPoint[2]>-5 or leftPoint[2]<-100):
                leftRect =[[-5.125,0,-20],[-3.125,0,-20],[-3.125,5,-20],[-5.125,5,-20],]
                rightRect=[[3.125,0,-20],[5.125,0,-20],[5.125,5,-20],[3.125,5,-20]]
            
            xAdjust = float(random.randint(-50,50))
            yAdjust = float(random.randint(-50,50))
            zAdjust = float(random.randint(-50,50))
            if(xAdjust == 0):
                xAdjust+=1
            if(yAdjust == 0):
                yAdjust+=1
            if(zAdjust == 0):
                zAdjust+=1
            rightRect, leftRect = addToRow(rightRect,leftRect, 0, 1.0/xAdjust)
            rightRect, leftRect = addToRow(rightRect,leftRect, 1, 1.0/yAdjust)
            rightRect, leftRect = addToRow(rightRect,leftRect, 2, 1.0/zAdjust)

            

            pygame.display.flip()

            pygame.time.wait(100)
            glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)         
    except Exception as e:
        %tb
        print (e)
main()

[  4.125   2.5   -20.   ]
[160.0, 92.01750552654266, 148.474130153656, 72.42640972137451, -4.125, 2.5, -20.0, 4.125, 2.5, -20.0]
[  3.79166667   2.47916667 -19.88888889]
[155.14459855063666, 89.47347747879988, 149.3035945679222, 72.83102653546041, -4.458333333333333, 2.479166666666667, -19.888888888888889, 3.7916666666666665, 2.479166666666667, -19.888888888888889]
[  3.73903509   2.50297619 -19.63888889]
[154.3063884738799, 88.26646726971285, 151.20420609707543, 73.75815417169683, -4.5109649122807021, 2.5029761904761907, -19.638888888888889, 3.7390350877192984, 2.5029761904761907, -19.638888888888889]
[  3.70455233   2.53238796 -19.30555556]
[153.69062171008324, 86.89381511495932, 153.81492620234866, 75.0316762581146, -4.5454476709013907, 2.5323879551820729, -19.305555555555557, 3.7045523290986084, 2.5323879551820729, -19.305555555555557]
[  3.60455233   2.56363796 -19.27852853]
[152.1790392427988, 85.83319299008508, 154.0305630006385, 75.13686494713258, -4.6454476709013912, 2.5636379

[  5.0873978    1.49902611 -18.62554606]
[174.96933570430855, 103.74603080378408, 159.430633258454, 77.77104573412893, -3.1626021983151311, 1.4990261073509885, -18.62554605964133, 5.087397801684868, 1.4990261073509885, -18.62554605964133]
[  5.05513974   1.43235944 -18.64595422]
[174.4517521520679, 104.05301752037515, 159.25613500783444, 77.68592463066156, -3.1948602628312606, 1.4323594406843219, -18.645954222906635, 5.055139737168739, 1.4323594406843219, -18.645954222906635]
[  5.13206281   1.49902611 -18.623732  ]
[175.66559037336748, 104.08989774269048, 159.44616272205766, 77.77862108272704, -3.1179371859081835, 1.4990261073509885, -18.623732000684413, 5.1320628140918156, 1.4990261073509885, -18.623732000684413]
[  5.24317393   1.47728698 -18.59595422]
[177.4199864215363, 105.07109117502203, 159.6843360377436, 77.8948031955888, -3.0068260747970728, 1.4772869769162058, -18.595954222906634, 5.2431739252029264, 1.4772869769162058, -18.595954222906634]
[  5.27258569   1.4545597  -17.595

KeyboardInterrupt: 